In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
tracker = None
bbox = None
tracking = False

# Optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Tracking variables
prev_frame = None
prev_pts = None
tracking_history = []  # Store recent tracking positions

# Select ROI
ret, frame = cap.read()
cv2.putText(frame, "Select ROI & press ENTER", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select ROI")

if bbox != (0, 0, 0, 0):
    # Initialize CSRT tracker
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    tracking = True
    
    # Initialize optical flow
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    x, y, w, h = map(int, bbox)
    roi = prev_frame[y:y+h, x:x+w]
    prev_pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
    if prev_pts is not None:
        prev_pts = prev_pts + np.array([x, y], dtype=np.float32)  # Adjust coordinates to global frame

def update_tracker_with_optical_flow(frame, gray_frame):
    global prev_frame, prev_pts, bbox
    
    if prev_frame is None or prev_pts is None or len(prev_pts) < 5:
        return False, bbox
    
    # Calculate optical flow
    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)
    
    # Filter good points
    if next_pts is not None:
        good_new = next_pts[status == 1]
        good_old = prev_pts[status == 1]
        
        if len(good_new) > 5:  # Enough points to estimate movement
            # Calculate bounding box movement
            movement = np.mean(good_new - good_old, axis=0)
            x, y, w, h = bbox
            
            # Update bounding box position
            new_x = x + movement[0]
            new_y = y + movement[1]
            
            # Update previous points for next iteration
            prev_pts = good_new.reshape(-1, 1, 2)
            prev_frame = gray_frame.copy()
            
            # Update bbox
            bbox = (new_x, new_y, w, h)
            return True, bbox
    
    # Not enough points to track
    return False, bbox

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if tracking and tracker:
        # 1. Try CSRT tracker
        tracking_success, new_bbox = tracker.update(frame)
        
        # 2. Apply optical flow
        of_success, of_bbox = update_tracker_with_optical_flow(frame, gray_frame)
            
        # Decide which result to use
        if tracking_success and of_success:
            # Blend CSRT and optical flow results (weighted average)
            csrt_weight = 0.7
            of_weight = 0.3
            x1, y1, w1, h1 = new_bbox
            x2, y2, w2, h2 = of_bbox
            
            blended_x = x1 * csrt_weight + x2 * of_weight
            blended_y = y1 * csrt_weight + y2 * of_weight
            bbox = (blended_x, blended_y, w1, h1)  # Keep original width/height
            
            # Update optical flow points occasionally for long-term stability
            if len(tracking_history) % 10 == 0:
                x, y, w, h = map(int, bbox)
                roi = gray_frame[max(0, y):min(frame.shape[0], y+h), 
                               max(0, x):min(frame.shape[1], x+w)]
                if roi.size > 0:
                    pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
                    if pts is not None and pts.size > 0:
                        prev_pts = pts + np.array([x, y], dtype=np.float32)
                        prev_frame = gray_frame.copy()
        elif tracking_success:
            bbox = new_bbox
        elif of_success:
            bbox = of_bbox
        
        # Store tracking history
        if tracking_success or of_success:
            x, y, w, h = map(float, bbox)
            center_x, center_y = x + w/2, y + h/2
            tracking_history.append((center_x, center_y))
            if len(tracking_history) > 30:  # Keep last 30 positions
                tracking_history.pop(0)
            
            print("Tracking: Success")
        else:
            print("Tracking: Lost")
        
    # Draw tracking information on frame
    if bbox is not None:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Draw tracking history
        if len(tracking_history) > 1:
            for i in range(1, len(tracking_history)):
                pt1 = tuple(map(int, tracking_history[i-1]))
                pt2 = tuple(map(int, tracking_history[i]))
                cv2.line(frame, pt1, pt2, (0, 0, 255), 1)
    
    # Display the frame
    cv2.imshow("Tracking", frame)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture(0)
tracker = None
bbox = None
tracking = False

# Optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Kalman Filter setup
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, vx, vy), 2 measurement variables (x, y)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],  [0, 1, 0, 1],  [0, 0, 1, 0],  [0, 0, 0, 1]], np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],  [0, 1, 0, 0]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.3
kalman.statePre = np.zeros((4, 1), np.float32)
kalman.statePost = np.zeros((4, 1), np.float32)

# Tracking variables
prev_frame = None
prev_pts = None

# Select ROI
ret, frame = cap.read()
cv2.putText(frame, "Select ROI & press ENTER", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select ROI")

if bbox != (0, 0, 0, 0):
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    tracking = True
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    x, y, w, h = map(int, bbox)
    roi = prev_frame[y:y+h, x:x+w]
    prev_pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
    if prev_pts is not None:
        prev_pts = prev_pts + np.array([x, y], dtype=np.float32)
    
    # Initialize Kalman Filter
    kalman.statePre[:2] = np.array([[x + w / 2], [y + h / 2]], np.float32)
    kalman.statePost = kalman.statePre.copy()

def update_tracker_with_optical_flow(frame, gray_frame):
    global prev_frame, prev_pts, bbox
    if prev_frame is None or prev_pts is None or len(prev_pts) < 5:
        return False, bbox
    
    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)
    if next_pts is not None:
        good_new = next_pts[status == 1]
        good_old = prev_pts[status == 1]
        if len(good_new) > 5:
            movement = np.mean(good_new - good_old, axis=0)
            x, y, w, h = bbox
            new_x = x + movement[0]
            new_y = y + movement[1]
            prev_pts = good_new.reshape(-1, 1, 2)
            prev_frame = gray_frame.copy()
            bbox = (new_x, new_y, w, h)
            return True, bbox
    return False, bbox

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if tracking:
        # Predict next position using Kalman Filter
        prediction = kalman.predict()
        pred_x, pred_y = prediction[0], prediction[1]
        
        # Update tracker
        tracking_success, new_bbox = tracker.update(frame)
        of_success, of_bbox = update_tracker_with_optical_flow(frame, gray_frame)
        
        if tracking_success and of_success:
            x1, y1, w1, h1 = new_bbox
            x2, y2, w2, h2 = of_bbox
            measured_x = (x1 + x2) / 2
            measured_y = (y1 + y2) / 2
        elif tracking_success:
            measured_x, measured_y = new_bbox[0], new_bbox[1]
        elif of_success:
            measured_x, measured_y = of_bbox[0], of_bbox[1]
        else:
            measured_x, measured_y = pred_x, pred_y
        
        measurement = np.array([[np.float32(measured_x)], [np.float32(measured_y)]])
        kalman.correct(measurement)
        bbox = (measured_x, measured_y, bbox[2], bbox[3])
        
        x, y, w, h = map(int, bbox)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\gemy2\AppData\Local\Temp\ipykernel_24648\3337927674.py:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y, w, h = map(int, bbox)


In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
tracker = None
bbox = None
tracking = False

# Optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Tracking variables
prev_frame = None
prev_pts = None
tracking_history = []  # Store recent tracking positions

# Select ROI
ret, frame = cap.read()
cv2.putText(frame, "Select ROI & press ENTER", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select ROI")

if bbox != (0, 0, 0, 0):
    # Initialize CSRT tracker
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    tracking = True
    
    # Initialize optical flow
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    x, y, w, h = map(int, bbox)
    roi = prev_frame[y:y+h, x:x+w]
    prev_pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
    if prev_pts is not None:
        prev_pts = prev_pts + np.array([x, y], dtype=np.float32)  # Adjust coordinates to global frame
    print('bbox:', bbox)
    print('roi:', roi)
    print('prev_pts:', prev_pts)

def update_tracker_with_optical_flow(frame, gray_frame):
    global prev_frame, prev_pts, bbox
    
    if prev_frame is None or prev_pts is None or len(prev_pts) < 5:
        return False, bbox
    
    # Calculate optical flow
    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)
    
    # Filter good points
    if next_pts is not None:
        good_new = next_pts[status == 1]
        good_old = prev_pts[status == 1]
        
        if len(good_new) > 5:  # Enough points to estimate movement
            # Calculate bounding box movement
            movement = np.mean(good_new - good_old, axis=0)
            x, y, w, h = bbox
            
            # Update bounding box position
            new_x = x + movement[0]
            new_y = y + movement[1]
            
            # Update previous points for next iteration
            prev_pts = good_new.reshape(-1, 1, 2)
            prev_frame = gray_frame.copy()
            
            # Update bbox
            bbox = (new_x, new_y, w, h)
            return True, bbox
    
    # Not enough points to track
    return False, bbox

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if tracking and tracker:
        # 1. Try CSRT tracker
        tracking_success, new_bbox = tracker.update(frame)
        
        # 2. Apply optical flow
        of_success, of_bbox = update_tracker_with_optical_flow(frame, gray_frame)
            
        # Decide which result to use
        if tracking_success and of_success:
            # Blend CSRT and optical flow results (weighted average)
            csrt_weight = 0.7
            of_weight = 0.3
            x1, y1, w1, h1 = new_bbox
            x2, y2, w2, h2 = of_bbox
            
            blended_x = x1 * csrt_weight + x2 * of_weight
            blended_y = y1 * csrt_weight + y2 * of_weight
            bbox = (blended_x, blended_y, w1, h1)  # Keep original width/height
            
            # Update optical flow points occasionally for long-term stability
            if len(tracking_history) % 10 == 0:
                x, y, w, h = map(int, bbox)
                roi = gray_frame[max(0, y):min(frame.shape[0], y+h), max(0, x):min(frame.shape[1], x+w)]
                if roi.size > 0:
                    pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
                    if pts is not None and pts.size > 0:
                        prev_pts = pts + np.array([x, y], dtype=np.float32)
                        prev_frame = gray_frame.copy()
        elif tracking_success:
            bbox = new_bbox
        elif of_success:
            bbox = of_bbox
        
        # Store tracking history
        if tracking_success or of_success:
            x, y, w, h = map(float, bbox)
            center_x, center_y = x + w/2, y + h/2
            tracking_history.append((center_x, center_y))
            if len(tracking_history) > 30:  # Keep last 30 positions
                tracking_history.pop(0)
            
            print("Tracking: Success")
        else:
            print("Tracking: Lost")
        
    # Draw tracking information on frame
    if bbox is not None:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Draw tracking history
        if len(tracking_history) > 1:
            for i in range(1, len(tracking_history)):
                pt1 = tuple(map(int, tracking_history[i-1]))
                pt2 = tuple(map(int, tracking_history[i]))
                cv2.line(frame, pt1, pt2, (0, 0, 255), 1)
    
    # Display the frame
    cv2.imshow("Tracking", frame)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()

bbox: (251, 362, 88, 63)
roi: [[137 135 134 ... 119 116 110]
 [138 137 136 ... 114 115 115]
 [138 137 137 ... 111 111 110]
 ...
 [119 118 116 ... 170 173 177]
 [117 116 115 ... 169 172 175]
 [117 115 113 ... 170 171 173]]
prev_pts: [[[305. 418.]]

 [[316. 368.]]

 [[327. 403.]]

 [[298. 416.]]

 [[312. 420.]]

 [[290. 413.]]]
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracki

In [8]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize tracking variables
tracker = None
bbox = None
tracking = False

# Optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, 
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Tracking variables
prev_frame = None
prev_pts = None
tracking_history = []  # Store recent tracking positions

# Kalman Filter setup
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, dx, dy), 2 measurement variables (x, y)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)  # Measurement matrix
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)  # State transition matrix
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03  # Process noise
kalman.measurementNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 0.1  # Measurement noise

# Select ROI
ret, frame = cap.read()
cv2.putText(frame, "Select ROI & press ENTER", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
bbox = cv2.selectROI("Select ROI", frame, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select ROI")

if bbox != (0, 0, 0, 0):
    # Initialize CSRT tracker
    tracker = cv2.TrackerCSRT_create()
    tracker.init(frame, bbox)
    tracking = True
    
    # Initialize optical flow
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    x, y, w, h = map(int, bbox)
    roi = prev_frame[y:y+h, x:x+w]
    prev_pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
    if prev_pts is not None:
        prev_pts = prev_pts + np.array([x, y], dtype=np.float32)  # Adjust coordinates to global frame
    
    # Initialize Kalman filter with current position
    center_x, center_y = x + w/2, y + h/2
    kalman.statePre = np.array([[center_x], [center_y], [0], [0]], np.float32)
    kalman.statePost = np.array([[center_x], [center_y], [0], [0]], np.float32)

def update_tracker_with_optical_flow(frame, gray_frame):
    global prev_frame, prev_pts, bbox
    
    if prev_frame is None or prev_pts is None or len(prev_pts) < 5:
        return False, bbox
    
    # Calculate optical flow
    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_frame, gray_frame, prev_pts, None, **lk_params)
    
    # Filter good points
    if next_pts is not None:
        good_new = next_pts[status == 1]
        good_old = prev_pts[status == 1]
        
        if len(good_new) > 5:  # Enough points to estimate movement
            # Calculate bounding box movement
            movement = np.mean(good_new - good_old, axis=0)
            x, y, w, h = bbox
            
            # Update bounding box position
            new_x = x + movement[0]
            new_y = y + movement[1]
            
            # Update previous points for next iteration
            prev_pts = good_new.reshape(-1, 1, 2)
            prev_frame = gray_frame.copy()
            
            # Update bbox
            bbox = (new_x, new_y, w, h)
            return True, bbox
    
    # Not enough points to track
    return False, bbox

def apply_kalman_filter(bbox):
    # Predict
    prediction = kalman.predict()
    
    # Get the current center from bbox
    x, y, w, h = bbox
    center_x, center_y = x + w/2, y + h/2
    
    # Correct with measurement
    measurement = np.array([[center_x], [center_y]], np.float32)
    kalman.correct(measurement)
    
    # Get the filtered position
    filtered_x = kalman.statePost[0][0]
    filtered_y = kalman.statePost[1][0]
    
    # Calculate the new bbox position
    new_x = filtered_x - w/2
    new_y = filtered_y - h/2
    
    return (new_x, new_y, w, h)

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if tracking and tracker:
        # 1. Try CSRT tracker
        tracking_success, new_bbox = tracker.update(frame)
        
        # 2. Apply optical flow
        of_success, of_bbox = update_tracker_with_optical_flow(frame, gray_frame)
            
        # Decide which result to use
        if tracking_success and of_success:
            # Blend CSRT and optical flow results (weighted average)
            csrt_weight = 0.7
            of_weight = 0.3
            x1, y1, w1, h1 = new_bbox
            x2, y2, w2, h2 = of_bbox
            
            blended_x = x1 * csrt_weight + x2 * of_weight
            blended_y = y1 * csrt_weight + y2 * of_weight
            bbox = (blended_x, blended_y, w1, h1)  # Keep original width/height
            
            # Apply Kalman filter to smooth the result
            bbox = apply_kalman_filter(bbox)
            
            # Update optical flow points occasionally for long-term stability
            if len(tracking_history) % 10 == 0:
                x, y, w, h = map(int, bbox)
                roi = gray_frame[max(0, y):min(frame.shape[0], y+h), 
                               max(0, x):min(frame.shape[1], x+w)]
                if roi.size > 0:
                    pts = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
                    if pts is not None and pts.size > 0:
                        prev_pts = pts + np.array([x, y], dtype=np.float32)
                        prev_frame = gray_frame.copy()
        elif tracking_success:
            bbox = new_bbox
            # Apply Kalman filter to smooth the result
            bbox = apply_kalman_filter(bbox)
        elif of_success:
            bbox = of_bbox
            # Apply Kalman filter to smooth the result
            bbox = apply_kalman_filter(bbox)
        
        # Store tracking history
        if tracking_success or of_success:
            x, y, w, h = map(float, bbox)
            center_x, center_y = x + w/2, y + h/2
            tracking_history.append((center_x, center_y))
            if len(tracking_history) > 30:  # Keep last 30 positions
                tracking_history.pop(0)
            
            print("Tracking: Success")
        else:
            print("Tracking: Lost")
        
    # Draw tracking information on frame
    if bbox is not None:
        x, y, w, h = map(int, bbox)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Draw tracking history
        if len(tracking_history) > 1:
            for i in range(1, len(tracking_history)):
                pt1 = tuple(map(int, tracking_history[i-1]))
                pt2 = tuple(map(int, tracking_history[i]))
                cv2.line(frame, pt1, pt2, (0, 0, 255), 1)
    
    # Display the frame
    cv2.imshow("Tracking", frame)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()

Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: Success
Tracking: 

In [37]:
# CSRT Tracker & Optical Flow & Kalman Filter  & Tracking History
import cv2
import numpy as np
from collections import deque

cap = cv2.VideoCapture(0)
tracker = cv2.TrackerCSRT_create()
ret, frame = cap.read()
bbox = cv2.selectROI("Select Object", frame, False)
tracker.init(frame, bbox)

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],[0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],[0, 1, 0, 1],[0, 0, 1, 0],[0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03


tracking_history = deque(maxlen=10)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    success, bbox = tracker.update(frame)

    if success:
        x, y, w, h = [int(v) for v in bbox]
        center_x, center_y = x + w // 2, y + h // 2

        measurement = np.array([[np.float32(center_x)], [np.float32(center_y)]])
        kalman.correct(measurement)

        tracking_history.append((center_x, center_y, w, h))

    else:
        
        if len(tracking_history) > 1:
            # الحصول على آخر موقعين معروفين
            p0 = np.array([[tracking_history[-2][:2]]], dtype=np.float32)
            p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, p0, None)

            if p1 is not None and st[0] == 1:
                # Optical Flow
                pred_x, pred_y = int(p1[0][0][0]), int(p1[0][0][1])
            else:
                #  Kalman Filter 
                prediction = kalman.predict()
                pred_x, pred_y = int(prediction[0]), int(prediction[1])

            # إعادة إنشاء التراكر عند الموقع المتوقع
            bbox = (pred_x - w // 2, pred_y - h // 2, w, h)
            tracker = cv2.TrackerCSRT_create()
            tracker.init(frame, bbox)

    prev_gray = gray.copy()

    if len(tracking_history) > 0:
        last_x, last_y, last_w, last_h = tracking_history[-1]
        cv2.rectangle(frame, (last_x - last_w // 2, last_y - last_h // 2),
                      (last_x + last_w // 2, last_y + last_h // 2), (0, 255, 0), 2)

    for i in range(1, len(tracking_history)):
        cv2.line(frame, tracking_history[i - 1][:2], tracking_history[i][:2], (0, 0, 255), 2)

    cv2.imshow("Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [24]:
import cv2
import numpy as np
from collections import deque

# Initialize webcam
cap = cv2.VideoCapture(0)

# Read first frame
ret, frame = cap.read()

# Let user select ROI (Region of Interest)
bbox = cv2.selectROI("Select Object to Track", frame, False)
cv2.destroyWindow("Select Object to Track")

# Initialize the CSRT tracker
tracker = cv2.legacy.TrackerCSRT_create()
tracker.init(frame, bbox)

# Initialize variables for tracking
tracking_active = True
bbox_history = deque(maxlen=10)  # Store last 10 positions
bbox_history.append(bbox)

# Initialize Kalman filter
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03

# Initialize variables for optical flow
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
old_points = None
    
while True:
    # Read a new frame
    ret, frame = cap.read()
        
    # Create a copy of the frame for visualization
    frame_display = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 1. CSRT Tracker
    if tracking_active:
        success, bbox = tracker.update(frame)
        
        # If tracking is successful, update history and Kalman filter
        if success:
            x, y, w, h = [int(v) for v in bbox]
            bbox_history.append((x, y, w, h))
            
            # Kalman measurement update
            kalman.correct(np.array([[x + w/2], [y + h/2]], np.float32))
            
            # Save points for optical flow
            if old_points is None or len(old_points) < 10:
                # Define region for feature detection
                roi = old_gray[y:y+h, x:x+w]
                # Find good features to track
                try:
                    points = cv2.goodFeaturesToTrack(roi, 100, 0.01, 10)
                    if points is not None:
                        old_points = points + np.array([x, y], np.float32)
                except cv2.error:
                    old_points = None
            
            # Draw the bbox
            cv2.rectangle(frame_display, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame_display, "Tracking", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            tracking_active = False
            
    # 2. If tracking fails, use prediction and attempt reinitialization
    if not tracking_active:
        # Predict using Kalman filter
        prediction = kalman.predict()
        pred_x, pred_y = int(prediction[0]), int(prediction[1])
        
        # Use the last known size from history
        if bbox_history:
            last_x, last_y, last_w, last_h = bbox_history[-1]
            predicted_bbox = (pred_x - last_w//2, pred_y - last_h//2, last_w, last_h)
            
            # Attempt to reinitialize tracker at the predicted position
            tracker = cv2.legacy.TrackerCSRT_create()
            success = tracker.init(frame, predicted_bbox)
            if success:
                tracking_active = True
                bbox = predicted_bbox
            
            # Draw the predicted position
            cv2.rectangle(frame_display, (pred_x - last_w//2, pred_y - last_h//2), 
                            (pred_x + last_w//2, pred_y + last_h//2), (0, 0, 255), 2)
            cv2.putText(frame_display, "Predicted", (pred_x - last_w//2, pred_y - last_h//2 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # 3. Lucas-Kanade Optical Flow for additional tracking
    if old_points is not None and len(old_points) > 0:
        new_points, status, _ = cv2.calcOpticalFlowPyrLK(old_gray, gray, old_points, None)
        
        if new_points is not None:
            # Keep only good points
            good_new = new_points[status == 1]
            good_old = old_points[status == 1]
            
            # If we have enough points, use them to refine the bbox
            if len(good_new) > 5 and tracking_active:
                # Calculate the median shift
                shifts = good_new - good_old
                if len(shifts) > 0:
                    median_shift = np.median(shifts, axis=0)
                    x, y, w, h = [int(v) for v in bbox]
                    
                    # Apply the shift to the current bbox
                    shifted_bbox = (x + int(median_shift[0]), y + int(median_shift[1]), w, h)
                    
                    # Update Kalman with optical flow information
                    center_x = shifted_bbox[0] + shifted_bbox[2]//2
                    center_y = shifted_bbox[1] + shifted_bbox[3]//2
                    kalman.correct(np.array([[center_x], [center_y]], np.float32))
                
            # Draw optical flow tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel().astype(int)
                c, d = old.ravel().astype(int)
                cv2.line(frame_display, (a, b), (c, d), (0, 255, 255), 2)
                cv2.circle(frame_display, (a, b), 3, (0, 255, 255), -1)
            
            # Update old_points for next frame
            old_points = good_new.reshape(-1, 1, 2)
    
    # Update old_gray for next optical flow calculation
    old_gray = gray.copy()
    
    # Draw tracking history
    for i in range(1, len(bbox_history)):
        prev_x, prev_y = bbox_history[i-1][0] + bbox_history[i-1][2]//2, bbox_history[i-1][1] + bbox_history[i-1][3]//2
        curr_x, curr_y = bbox_history[i][0] + bbox_history[i][2]//2, bbox_history[i][1] + bbox_history[i][3]//2
        cv2.line(frame_display, (prev_x, prev_y), (curr_x, curr_y), (255, 0, 0), 1)
    
    # Show the result
    cv2.imshow("Object Tracking", frame_display)
    
    # Break on ESC key
    if cv2.waitKey(1) == 27:  # ESC key
        break
        
cap.release()
cv2.destroyAllWindows()


C:\Users\gemy2\AppData\Local\Temp\ipykernel_4752\3232810261.py:76: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(prediction[0]), int(prediction[1])
